In [149]:
import numpy as np 

In [150]:
d = 2 
rho = np.random.randn(d,d) + 1j*np.random.randn(d,d)
rho = rho @ rho.T.conj()
rho = rho / np.trace( rho )
rho 

array([[0.38517531+0.j        , 0.07843788-0.21953638j],
       [0.07843788+0.21953638j, 0.61482469+0.j        ]])

In [151]:
paulis = np.array( 
        [ np.eye(2),
        np.array([0,1,1,0]).reshape(2,2),
        np.array([0,-1j,1j,0]).reshape(2,2),
        np.diag([1,-1])
        ])

In [152]:
def rho_param( y, z ):
    y = y / np.sum(y) 
    z = np.sqrt(y) * z / np.linalg.norm(z,axis=0)
    return z @ z.T.conj()

In [153]:
def y_z_drawing(alpha=1/4):
    y = np.random.gamma( alpha, 1, size=2 )
    z = np.random.randn(d,d) + 1j*np.random.randn(d,d)
    return [ y, z ]

In [154]:
def log_pseudo_likelihood( y, z, rho_ls, alpha =1/4 ):
        L = ( - 0.5 * 4 * np.linalg.norm(  rho_param(y,z) - rho_ls  )**2
                        + np.sum( alpha*np.log(y) - y ) 
                        )
        return L 

In [155]:
y, z = y_z_drawing()
rho = rho_param( y, z )
y = np.ones(2)/2
z = np.eye(2)

# y, z = y_z_drawing()

In [156]:
measures = np.array( [ np.trace(rho@pauli) for pauli in paulis ] )
rho_ls = ( paulis.reshape(4,-1).T @ measures ).reshape(2,2)/2

In [157]:
rho

array([[0.1351099+0.j        , 0.0341672+0.31800157j],
       [0.0341672-0.31800157j, 0.8648901+0.j        ]])

In [158]:
rho_param( y, z )

array([[0.5, 0. ],
       [0. , 0.5]])

In [159]:
rhos = []
for k in range(1000):
    beta_y = 0.1
    beta_z = 0.1
    alpha = 1/4 
    # y, z = y_z_drawing()
    logX = log_pseudo_likelihood( y, z, rho_ls )

    for j in range(100):
        eta_k = np.random.randn(d)
        xi_k  = np.random.randn(d,d) + 1j*np.random.randn(d,d)

        y_hat = y*np.exp(beta_y*eta_k)
        z_hat = np.sqrt( 1 - beta_z**2 )*z + beta_z*xi_k
        logY  = log_pseudo_likelihood( y_hat, z_hat, rho_ls ) 

        log_prob = min( 0, logY - logX ) 
        # print( np.exp(logY), np.exp(log_prob) )
        if np.log(np.random.rand()) < log_prob:
            y = y_hat
            z = z_hat
            logX = logY 

    rhos.append( rho_param( y, z ) )

In [160]:
np.mean( rhos, axis=0 )

array([[0.32177171+0.j        , 0.01171094+0.14521125j],
       [0.01171094-0.14521125j, 0.67822829+0.j        ]])

In [161]:
rho 

array([[0.1351099+0.j        , 0.0341672+0.31800157j],
       [0.0341672-0.31800157j, 0.8648901+0.j        ]])